In [5]:
#!/usr/bin/env python3
# star_network_identify.py
# ---------------------------------------------------------------
"""
Identify star-like network topologies (A/B/C) from time-series produced by a
high-precision coupled map lattice (CML). The workflow is:

1) Simulate node dynamics on directed star graphs with high numerical precision.
2) Compute node-wise error strengths ‖x_{t+1} - 2 x_t‖ (wrapped on the circle).
3) Use a 2-component Gaussian Mixture Model (GMM) to separate hubs vs. leaves.
4) With two segments (from B/C candidates), compare hubs' residual variances to
   decide whether a segment came from type-B (both hubs fully connected) or
   type-C (leaves split across two hubs).

Conventions
-----------
- Adjacency A is binary and directed. A[j, i] = 1 means a directed edge j → i,
  i.e., node j contributes to the coupling term of node i.
- Trajectories are stored as a NumPy array of shape (N, T_seg), with N nodes
  and T_seg time steps after discarding transients.
"""

from __future__ import annotations
from decimal import Decimal, getcontext
import numpy as np
import mpmath as mp
from typing import Callable, Tuple
from sklearn.mixture import GaussianMixture

# ---------- Global High Precision Settings ----------
getcontext().prec = 200
mp.mp.dps = getcontext().prec
TWOPI = mp.mpf('6.283185307179586476925286766559')
SIGMA_H2 = 0.5  # ∫ h^2(0,y) dm(y)  (kept for reference; not used directly)

# ------------------------------------------------------------------
#  I. High-Precision Network System (unchanged)
# ------------------------------------------------------------------
class GraphSystemDecimal:
    """
    Coupled map lattice on a directed graph with high-precision arithmetic.

    Each node evolves via a local map (default: doubling map x ↦ 2x mod 1),
    plus diffusive sinusoidal coupling from its in-neighbors.

    Parameters
    ----------
    A : np.ndarray
        Directed adjacency matrix of shape (N, N). A[j, i] = 1 indicates j → i.
    alpha : str, optional
        Coupling strength as a decimal string (for exact Decimal parsing).
    local_map : Callable[[Decimal], Decimal], optional
        Local map f(x). Defaults to doubling map `(2*x) % 1`.
    coupling_fn : Callable[[Decimal, Decimal], Decimal], optional
        Pairwise coupling c(x_s, x_t) from source s to target t. If None,
        uses a sinusoidal diffusive term `-sin(2π x_s) + sin(2π x_t)`.
    seed : int, optional
        Random seed for initial conditions.

    Attributes
    ----------
    N : int
        Number of nodes.
    Delta : float
        Maximum out-degree (max column sum) used for normalization.
    x : list[Decimal]
        Current node states.
    t : int
        Current time step.

    Notes
    -----
    - High precision is enforced via Python's Decimal and mpmath.
    - The coupling increment at node i is normalized by Delta.
    """

    def __init__(self, A: np.ndarray, alpha: str = '0.25',
                 local_map: Callable[[Decimal], Decimal] | None = None,
                 coupling_fn: Callable[[Decimal, Decimal], Decimal] | None = None,
                 seed: int = 0):
        self.A = np.asarray(A, dtype=float)
        self.N = self.A.shape[0]
        self.Delta = self.A.sum(axis=0).max()
        self.alpha = Decimal(alpha)
        self.local_map = local_map or (lambda x: (Decimal(2) * x) % 1)
        self.coupling = coupling_fn or self._default_coupling
        self.rng = np.random.default_rng(seed)
        self.reset()

    @staticmethod
    def _default_coupling(xs: Decimal, xt: Decimal) -> Decimal:
        """
        Default sinusoidal diffusive coupling.

        Parameters
        ----------
        xs : Decimal
            Source node state.
        xt : Decimal
            Target node state.

        Returns
        -------
        Decimal
            c(xs, xt) = -sin(2π xs) + sin(2π xt), as a Decimal.
        """
        v = -mp.sin(TWOPI * mp.mpf(str(xs))) + mp.sin(TWOPI * mp.mpf(str(xt)))
        return Decimal(str(v))

    def _coupling_term(self):
        """
        Compute normalized coupling increment for each node.

        Returns
        -------
        list[Decimal]
            A list of length N with the coupling increment for each node,
            normalized by the maximum out-degree Δ.

        Notes
        -----
        The increment for node i is the sum over j with A[j, i] = 1 of
        c(x_j, x_i), divided by Δ to keep scales comparable across graphs.
        """
        incr = [Decimal(0)] * self.N
        for j in range(self.N):
            if self.A[j].sum() == 0:
                continue  # node j has no outgoing edges
            for i in range(self.N):
                if self.A[j, i]:
                    incr[i] += self.coupling(self.x[j], self.x[i])
        d = Decimal(str(self.Delta))
        return [v / d for v in incr]

    def step(self):
        """
        Advance the system by one time step.

        Returns
        -------
        list[Decimal]
            The updated state vector x_{t+1} (length N) as Decimals.
        """
        xn = [self.local_map(x) for x in self.x]  # local map update
        coup = self._coupling_term()              # diffusive coupling
        xn = [(xi + self.alpha * ci) % 1 for xi, ci in zip(xn, coup)]
        self.x = xn
        self.t += 1
        return xn

    def reset(self):
        """
        Reset the system to a fresh random initial condition.

        Notes
        -----
        States are sampled i.i.d. ~ Uniform(0, 1) and stored as Decimal.
        """
        self.x = [Decimal(str(v)) for v in self.rng.random(self.N)]
        self.t = 0

    def run(self, T: int, discard: int = 0):
        """
        Simulate for T time steps and return the trajectory after discarding transients.

        Parameters
        ----------
        T : int
            Total number of steps to simulate.
        discard : int, optional
            Number of initial steps to discard as transients.

        Returns
        -------
        np.ndarray
            Array of shape (N, max(0, T - discard)) with float64 views of states.
        """
        traj = np.zeros((self.N, max(0, T - discard)))
        for k in range(T):
            xt = self.step()
            if k >= discard:
                traj[:, k - discard] = [float(v) for v in xt]
        return traj


# ------------------------------------------------------------------
#  II. Star Graph Generators (three variants)
# ------------------------------------------------------------------

def graph_A(N: int):
    """
    Create a star graph with a single hub (node N-1) pointed to by all leaves.

    Parameters
    ----------
    N : int
        Number of nodes.

    Returns
    -------
    np.ndarray
        Adjacency matrix A of shape (N, N) with A[j, N-1] = 1 for j = 0..N-2.
    """
    A = np.zeros((N, N))
    A[np.arange(N - 1), N - 1] = 1
    return A


def graph_B(N: int):
    """
    Create a star-like graph with two hubs (nodes N-2 and N-1).
    Every leaf connects to both hubs.

    Parameters
    ----------
    N : int
        Number of nodes.

    Returns
    -------
    np.ndarray
        Adjacency matrix A with A[leaf, N-1] = A[leaf, N-2] = 1 for all leaves.
    """
    A = np.zeros((N, N))
    leaves = np.arange(N - 2)
    A[leaves, N - 1] = 1
    A[leaves, N - 2] = 1
    return A


def graph_C(N: int):
    """
    Create a two-hub graph where leaves are split into two halves;
    each half connects to exactly one of the hubs.

    Parameters
    ----------
    N : int
        Number of nodes.

    Returns
    -------
    np.ndarray
        Adjacency matrix A with two hubs (N-2, N-1) and disjoint leaf sets.
    """
    A = np.zeros((N, N))
    half = N // 2
    A[np.arange(half - 1), N - 2] = 1
    A[np.arange(half - 1, N - 2), N - 1] = 1
    return A


# ------------------------------------------------------------------
#  III. GMM-based Hub Detection & Core Statistics
# ------------------------------------------------------------------

def moddiff(u):
    """
    Wrap a real array onto the interval (-0.5, 0.5] using modulo-1 arithmetic.

    Parameters
    ----------
    u : array_like
        Input values (can be scalar or array).

    Returns
    -------
    np.ndarray or float
        Wrapped values with the same shape as input.
    """
    return ((u + 0.5) % 1) - 0.5


def compute_strength(traj):
    """
    Compute node-wise mean error strength ‖x_{t+1} - 2 x_t‖ (wrapped).

    Parameters
    ----------
    traj : np.ndarray
        Trajectory of shape (N, T), after discarding transients.

    Returns
    -------
    np.ndarray
        Vector of length N, where entry i is the mean absolute wrapped error
        for node i across time.
    """
    x, x1 = traj[:, :-1], traj[:, 1:]
    return np.abs(moddiff(x1 - 2 * x)).mean(axis=1)


def gmm_hubs(S, seed=0):
    """
    Use a 2-component Gaussian Mixture Model to separate hubs (larger error
    strength) from leaves.

    Parameters
    ----------
    S : np.ndarray
        Mean error strengths for N nodes; shape (N,).
    seed : int, optional
        Random seed for GMM initialization.

    Returns
    -------
    np.ndarray
        Boolean mask of shape (N,), where True indicates a hub (the component
        with the larger mean).
    """
    g = GaussianMixture(2, random_state=seed).fit(S.reshape(-1, 1))
    return g.predict(S.reshape(-1, 1)) == np.argmax(g.means_)


def beta_var(x: np.ndarray) -> float:
    """
    Estimate the variance of residuals in
    y_t = x_{t+1} - 2 x_t + β sin(2π x_t), via least squares for β.

    Parameters
    ----------
    x : np.ndarray
        Single-node time series of shape (T,).

    Returns
    -------
    float
        Variance of residuals y_t + β sin(2π x_t).
    """
    y = moddiff(x[1:] - 2 * x[:-1])
    s = -np.sin(2 * np.pi * x[:-1])
    beta = -(y @ s) / (s @ s)
    resid = y + beta * s
    return resid.var()


# ------------------------------------------------------------------
#  IV-a  Single Segment → Classify A vs (B/C)
# ------------------------------------------------------------------

def classify_A_and_BC(traj: np.ndarray, N: int) -> str:
    """
    Classify a single segment as 'A_N' (exactly one hub) or 'B_N and C_N' (two hubs).

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T).
    N : int
        Number of nodes (kept for signature compatibility; not used).

    Returns
    -------
    str
        'A_N' if exactly one hub is detected; otherwise 'B_N and C_N'.
    """
    S = compute_strength(traj)
    hubs = np.where(gmm_hubs(S))[0]
    if hubs.size == 1:
        return "A_N"
    return "B_N and C_N"


# ------------------------------------------------------------------
#  IV-b  Compute "mean hub variance" for one segment
# ------------------------------------------------------------------

def average_hub_variance(traj: np.ndarray) -> float:
    """
    Compute the mean β-residual variance over the two hubs of a B/C star graph.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T).

    Returns
    -------
    float
        The average of beta_var over the two hubs.

    Raises
    ------
    RuntimeError
        If the segment does not appear to have exactly two hubs.
    """
    S = compute_strength(traj)
    hubs = np.where(gmm_hubs(S))[0]
    if hubs.size != 2:
        raise RuntimeError("This segment does not correspond to a B/C graph (number of hubs ≠ 2)")
    return float(np.mean([beta_var(traj[i]) for i in hubs]))

# ------------------------------------------------------------------
#  IV-c **Key addition**: Two segments → compare variances → classify B vs C
# ------------------------------------------------------------------
def classify_B_vs_C(traj_first: np.ndarray, traj_second: np.ndarray) -> Tuple[str, float, float]:
    """
    Distinguish type-B vs. type-C using hub residual variances from two segments.

    Parameters
    ----------
    traj_first : np.ndarray
        First trajectory, shape (N, T), from a B/C candidate graph.
    traj_second : np.ndarray
        Second trajectory, shape (N, T), from a B/C candidate graph.

    Returns
    -------
    tuple[str, float, float]
        (label, var_first, var_second)
        - label: 'first_is_B' if var_first < var_second, else 'first_is_C'
        - var_first: mean hub variance of the first segment
        - var_second: mean hub variance of the second segment

    Notes
    -----
    Lower hub variance ⇒ higher in-degree ⇒ type-B.
    """
    var1 = average_hub_variance(traj_first)
    var2 = average_hub_variance(traj_second)
    if var1 < var2:  # smaller variance ⇒ larger in-degree ⇒ B graph
        return "first_is_B", var1, var2
    else:
        return "first_is_C", var1, var2

# ------------------------------------------------------------------
#  V. Demo
# ------------------------------------------------------------------
if __name__ == "__main__":
    N, T, discard = 10, 6000, 600
    alpha = '0.25'

    # 1) Demonstrate single-segment classification A/B/C
    for gname, maker in [("A_N", graph_A), ("B_N", graph_B), ("C_N", graph_C)]:
        traj = GraphSystemDecimal(maker(N), alpha=alpha, seed=hash(gname) % 2**32).run(T, discard)
        print(f"{gname}  → classify_ABC → {classify_A_and_BC(traj, N)}")

    # 2) Demonstrate variance comparison to distinguish B / C
    trajB = GraphSystemDecimal(graph_B(N), alpha=alpha, seed=1).run(T, discard)
    trajC = GraphSystemDecimal(graph_C(N), alpha=alpha, seed=2).run(T, discard)

    res, v_first, v_second = classify_B_vs_C(trajB, trajC)  # B comes first
    print("\nComparing two sequences:", res)
    print(f"  Mean hub variance of the first segment  = {v_first:.6e}")
    print(f"  Mean hub variance of the second segment = {v_second:.6e}")


A_N  → classify_ABC → A_N
B_N  → classify_ABC → B_N and C_N
C_N  → classify_ABC → B_N and C_N

Comparing two sequences: first_is_B
  Mean hub variance of the first segment  = 3.902977e-03
  Mean hub variance of the second segment = 7.846020e-03


In [6]:
# ======== New or Replacement Section Begins =================================
# I. General Logistic Map (Decimal version, co-existing with original 2 x mod 1)
def logistic_map_decimal(x: Decimal) -> Decimal:
    """
    Logistic map in Decimal precision: f(x) = 4 x (1 - x)  (mod 1).

    Parameters
    ----------
    x : Decimal
        State value in [0, 1).

    Returns
    -------
    Decimal
        f(x) mapped back to [0, 1) using modulo-1, to match the original design.

    Notes
    -----
    Keeping the modulo-1 ensures consistency with the doubling-map implementation.
    """
    # “% 1” keeps the same format as in the original implementation
    return (Decimal(4) * x * (Decimal(1) - x)) % 1


# II. Example interface for an optional coupling function
def coupling_sin_diff(xs: Decimal, xt: Decimal) -> Decimal:
    """
    Sinusoidal diffusive coupling in Decimal precision:
    c(xs, xt) = -sin(2π xs) + sin(2π xt).

    Parameters
    ----------
    xs : Decimal
        Source node state.
    xt : Decimal
        Target node state.

    Returns
    -------
    Decimal
        Coupling contribution from xs to xt.
    """
    v = -mp.sin(TWOPI * mp.mpf(str(xs))) + mp.sin(TWOPI * mp.mpf(str(xt)))
    return Decimal(str(v))


# III. --- Modify compute_strength / beta_var so they depend on local_map ---
def compute_strength(
    traj: np.ndarray,
    local_map_vec: Callable[[np.ndarray], np.ndarray]
) -> np.ndarray:
    """
    Compute node-wise mean absolute innovation relative to the local map:
    S_i = ⟨|Δ_i|⟩, where Δ_i(t) = x_{t+1,i} − f(x_{t,i}) wrapped by modulo-1.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T) for N nodes and T time steps
        (after any transient discard).
    local_map_vec : Callable[[np.ndarray], np.ndarray]
        Vectorized local map f applied elementwise to traj[:, :-1].
        It must accept an array of shape (N, T-1) and return the same shape.

    Returns
    -------
    np.ndarray
        Strength vector S of shape (N,), one entry per node.
    """
    x, x1 = traj[:, :-1], traj[:, 1:]
    Delta = moddiff(x1 - local_map_vec(x))
    return np.abs(Delta).mean(axis=1)


def beta_var(
    traj_i: np.ndarray,
    local_map_vec: Callable[[np.ndarray], np.ndarray],
    I_h_vec: Callable[[np.ndarray], np.ndarray]
) -> float:
    """
    Estimate β by least squares and return the residual variance for one node.

    Model
    -----
    y_t = x_{t+1} − f(x_t) + β · I_h(x_t),
    where I_h(x) = ∫ h(x, y) dm(y).
    All differences are wrapped via modulo-1 to stay on the circle.

    Parameters
    ----------
    traj_i : np.ndarray
        Single-node series of shape (T,).
    local_map_vec : Callable[[np.ndarray], np.ndarray]
        Vectorized local map f for arrays of shape (T-1,) → (T-1,).
    I_h_vec : Callable[[np.ndarray], np.ndarray]
        Vectorized function I_h for arrays of shape (T-1,) → (T-1,).

    Returns
    -------
    float
        Variance of residuals y + β · I_h(x).

    Notes
    -----
    β is obtained by minimizing ‖y + β s‖² with s = I_h(x), yielding
    β* = −(yᵀ s)/(sᵀ s).
    """
    x     = traj_i[:-1]
    y     = moddiff(traj_i[1:] - local_map_vec(x))
    s     = I_h_vec(x)
    beta  = -(y @ s) / (s @ s)
    resid = y + beta * s
    return resid.var()


# IV. --- Vectorized utilities, isolated from the Decimal system -------------
# logistic_vec: elementwise version of f(x) = 4x(1-x) acting on ndarray inputs.
logistic_vec = np.vectorize(lambda u: 4.0 * u * (1.0 - u))          # f(x)

# Ih_vec: elementwise version of I_h(x) = ∫ h(x, y) dm(y); here chosen as -sin(2πx).
Ih_vec       = np.vectorize(lambda u: -np.sin(2 * np.pi * u))       # ∫ h dm


# V. --- Adapted classification functions -----------------------------------
def classify_A_and_BC(traj: np.ndarray) -> str:
    """
    Classify a single segment as 'A_N' (one hub) or 'B_N and C_N' (two hubs),
    using the logistic local map and sinusoidal integral I_h by default.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T).

    Returns
    -------
    str
        'A_N' if exactly one hub is detected by GMM; otherwise 'B_N and C_N'.
    """
    S    = compute_strength(traj, logistic_vec)
    hubs = np.where(gmm_hubs(S))[0]
    return "A_N" if hubs.size == 1 else "B_N and C_N"


def average_hub_variance(traj: np.ndarray) -> float:
    """
    Compute the mean residual variance across the two detected hubs
    for a B/C star graph, using the logistic local map and I_h.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T).

    Returns
    -------
    float
        Average of beta_var over the two hubs.

    Raises
    ------
    RuntimeError
        If the number of detected hubs is not exactly two.
    """
    S    = compute_strength(traj, logistic_vec)
    hubs = np.where(gmm_hubs(S))[0]
    if hubs.size != 2:
        raise RuntimeError("Data are not from a B/C graph (number of hubs ≠ 2)")
    vars_ = [beta_var(traj[i], logistic_vec, Ih_vec) for i in hubs]
    return float(np.mean(vars_))


# VI. ------------------------ Demo -----------------------------------------
if __name__ == "__main__":
    """
    Demo: run the system under the logistic map and sinusoidal coupling,
    then classify A/B/C for single segments and distinguish B vs C by variance.
    """
    N, T, discard = 10, 6000, 600
    alpha = '0.25'

    local_map = logistic_map_decimal
    coupling  = coupling_sin_diff

    # Demonstrate single-segment classification for A/B/C
    for gname, maker in [("A_N", graph_A),
                         ("B_N", graph_B),
                         ("C_N", graph_C)]:
        gs   = GraphSystemDecimal(maker(N), alpha=alpha,
                                  local_map=local_map,
                                  coupling_fn=coupling,
                                  seed=hash(gname) % 2**32)
        traj = gs.run(T, discard)
        print(f"{gname}  → classify_ABC → {classify_A_and_BC(traj)}")

    # Compare two sequences to distinguish between B and C
    trajB = GraphSystemDecimal(graph_B(N), alpha=alpha,
                               local_map=local_map,
                               coupling_fn=coupling,
                               seed=1).run(T, discard)
    trajC = GraphSystemDecimal(graph_C(N), alpha=alpha,
                               local_map=local_map,
                               coupling_fn=coupling,
                               seed=2).run(T, discard)

    res, v_first, v_second = classify_B_vs_C(trajB, trajC)
    print("\nCompare the two sequences:", res)
    print(f"  Mean hub variance for the first sequence  = {v_first:.6e}")
    print(f"  Mean hub variance for the second sequence = {v_second:.6e}")
# ======== New or Replacement Section Ends ===================================


A_N  → classify_ABC → A_N
B_N  → classify_ABC → B_N and C_N
C_N  → classify_ABC → B_N and C_N

Compare the two sequences: first_is_B
  Mean hub variance for the first sequence  = 2.936251e-03
  Mean hub variance for the second sequence = 6.113247e-03


In [7]:
# ---------- 1. Theoretical variance function ---------------------
def theoretical_hub_variance(N: int,
                             graph_type: str,
                             sigma_h2: float = 0.3898615457,
                             alpha: float = 0.25) -> float:
    """
    Compute the closed-form (idealized) variance of hub nodes for type-B or type-C stars.

    The model assumes a diffusive coupling with per-step innovation variance
    proportional to σ_h^2 and a normalization by the maximum out-degree Δ.
    For star variants:
      - B: every leaf connects to both hubs → L = N - 2, Δ = N - 2
      - C: leaves split evenly across the two hubs → L = N//2 - 1, Δ = N//2 - 1

    The returned variance is:
        Var_hub = (alpha**2) * sigma_h2 * L / (Delta**2)

    Parameters
    ----------
    N : int
        Number of nodes in the graph.
    graph_type : str
        'B' or 'C' for the corresponding star topology.
    sigma_h2 : float, optional
        The integral of squared kernel (e.g., σ_h^2 = ∫ h^2 dm) used by the theory.
    alpha : float, optional
        Coupling strength.

    Returns
    -------
    float
        Theoretical hub variance for the specified graph type.

    Raises
    ------
    ValueError
        If `graph_type` is not 'B' or 'C'.
    """
    if graph_type == "B":
        L = N - 2
        Delta = N - 2
    elif graph_type == "C":
        L = N // 2 - 1
        Delta = N // 2 - 1
    else:
        raise ValueError("graph_type must be 'B' or 'C'")
    return (alpha ** 2) * sigma_h2 * L / (Delta ** 2)


# ---------- 2. Single-sequence B/C classification -------------------
def classify_single_BC_theory(traj: np.ndarray, N: int) -> tuple[str, dict]:
    """
    Classify a single N×T trajectory as 'B_N' or 'C_N' by matching empirical vs. theoretical hub variances.

    Procedure
    ---------
    a) Detect hubs with a 2-component GMM on node strength S_i = ⟨|Δ_i|⟩ where
       Δ_i(t) = x_{t+1,i} − f(x_{t,i}), using the logistic local map f.
    b) Compute empirical hub variance V_hat as the mean of β-residual variances
       across the two hubs (see `average_hub_variance`).
    c) Compute theoretical hub variances V_B_th, V_C_th via `theoretical_hub_variance`.
    d) Pick the label minimizing |log V_hat − log V_th|.

    Parameters
    ----------
    traj : np.ndarray
        Trajectory array of shape (N, T), after any transient discard.
    N : int
        Number of nodes (used in the theoretical formulas).

    Returns
    -------
    tuple[str, dict]
        - label : {'B_N', 'C_N'}
            Predicted graph type for the given trajectory.
        - info : dict
            Debugging payload with keys:
              * 'V_hat'  : float, empirical mean hub variance
              * 'V_B_th' : float, theoretical hub variance for type-B
              * 'V_C_th' : float, theoretical hub variance for type-C
              * 'd_B'    : float, |log(V_hat) - log(V_B_th)|
              * 'd_C'    : float, |log(V_hat) - log(V_C_th)|
              * 'hubs'   : list[int], indices of detected hubs

    Raises
    ------
    RuntimeError
        If the detected number of hubs is not exactly two.

    Notes
    -----
    - This function relies on the global `logistic_vec`, `gmm_hubs`, and
      `average_hub_variance` utilities defined elsewhere.
    - Assumes V_hat, V_B_th, V_C_th > 0 so that logarithms are defined.
    """
    # a) Locate hubs
    S = compute_strength(traj, logistic_vec)          # Use a different mapper? Modify this call.
    hubs = np.where(gmm_hubs(S))[0]
    if hubs.size != 2:
        raise RuntimeError("Number of hubs in trajectory ≠ 2; not a B/C graph")

    # b) Empirical average variance of hubs
    V_hat = average_hub_variance(traj)

    # c) Theoretical values
    V_B_th = theoretical_hub_variance(N, "B")
    V_C_th = theoretical_hub_variance(N, "C")

    # d) Log-distance between empirical and theoretical variances
    d_B = abs(np.log(V_hat) - np.log(V_B_th))
    d_C = abs(np.log(V_hat) - np.log(V_C_th))

    label = "B_N" if d_B < d_C else "C_N"
    info = {
        "V_hat": V_hat,
        "V_B_th": V_B_th,
        "V_C_th": V_C_th,
        "d_B": d_B,
        "d_C": d_C,
        "hubs": hubs.tolist(),
    }
    return label, info


# ---------- 3. Demo --------------------------------
if __name__ == "__main__":
    """
    Demo
    ----
    Generate trajectories from type-B and type-C stars under the logistic map
    and sinusoidal diffusive coupling, then classify each single sequence using
    the theory-matching approach above.
    """
    N, T, discard = 50, 8000, 800
    alpha = 0.25
    local_map = logistic_map_decimal
    coupling  = coupling_sin_diff

    # Simulate B and C star graphs
    trajB = (GraphSystemDecimal(graph_B(N),
                                alpha=alpha,
                                local_map=local_map,
                                coupling_fn=coupling,
                                seed=1)
             .run(T, discard))
    trajC = (GraphSystemDecimal(graph_C(N),
                                alpha=alpha,
                                local_map=local_map,
                                coupling_fn=coupling,
                                seed=2)
             .run(T, discard))

    # Classify each trajectory independently
    resB, infoB = classify_single_BC_theory(trajB, N)
    resC, infoC = classify_single_BC_theory(trajC, N)

    print("Trajectory B → classified as:", resB, "| debug:", infoB)
    print("Trajectory C → classified as:", resC, "| debug:", infoC)


Trajectory B → classified as: B_N | debug: {'V_hat': 0.0005113619541740942, 'V_B_th': 0.0005076322209635416, 'V_C_th': 0.0010152644419270831, 'd_B': 0.0073204537525510815, 'd_C': 0.6858267268073943, 'hubs': [48, 49]}
Trajectory C → classified as: C_N | debug: {'V_hat': 0.0010205339499985673, 'V_B_th': 0.0005076322209635416, 'V_C_th': 0.0010152644419270831, 'd_B': 0.6983240387998242, 'd_C': 0.00517685823987879, 'hubs': [48, 49]}
